J'essaye de refaire la partie **7.1 Linear Gaussian Experiment** à partir du code de Yuyang Shi (https://github.com/yuyang-shi/MLMC_Gradients) et en suivant les étapes de l'algorithme 1.

In [7]:
!pip install torch

     -------------------------------------- 262.1/262.1 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pybrain: filename=PyBrain-0.3-py3-none-any.whl size=399012 sha256=387aba031ffcd23d64966fe58fa81cc2452831a70d7697c069be59deb646f267
  Stored in directory: c:\users\peign\appdata\local\pip\cache\wheels\61\2f\96\4efbe44e973e830f8425e0ea56e77aa1277fc24dc5a8c0ebe8
Successfully built pybrain


In [29]:
import numpy as np
import torch
from torch.distributions import MultivariateNormal
from torch.distributions import Independent, Normal

In [76]:
latent_dim = 20
size = 50
theta_true = torch.ones(latent_dim)

In [77]:
z = MultivariateNormal(theta_true, torch.eye(latent_dim)).sample((size,))
x = MultivariateNormal(z, torch.eye(latent_dim)).sample()

In [78]:
def compute_log_p_q(x, z):
    # définir les distributions p et q
    mu_p = torch.cat((x, z), dim=1)
    mu_q = z
    cov_p = torch.eye(x.shape[1] + z.shape[1])
    cov_p[:x.shape[1], :x.shape[1]] = torch.eye(x.shape[1])
    cov_q = torch.eye(z.shape[1])

    # calculer log(p)
    dist_p = MultivariateNormal(mu_p, cov_p)
    log_p = dist_p.log_prob(torch.cat((x, z), dim=-1)).view(-1, 2, 1, 1)
    # calculer log(q)
    dist_q = MultivariateNormal(mu_q, cov_q)
    log_q = dist_q.log_prob(z).view(-1, 2, 1, 1)
    
    return log_p, log_q

In [79]:
# Les paramètres à définir pour calculer l'estimateur
geom_prob = 0.6
K_cap = np.inf
I0_start_level = 0
K=1

def _iwae(log_w):
    return - logmeanexp(log_w, dim=-2)

In [80]:
log_p, log_q = compute_log_p_q(x, z)
log_w = log_p - log_q

In [81]:
def ss(log_w, K, I0_start_level, train_function=_iwae):
    #first step : generate K as a geometric random variable
    K_pmf = lambda k: geom_prob * (1 - geom_prob) ** k / (
                    1 - (1 - geom_prob) ** (K_cap + 1))
    #second step : compute log(w)
    log_w = log_w.view(log_w.shape[0], 2 ** (K + 1), 2 ** I0_start_level, 1)
    
    #third step : compute I0
    I0 = train_function(log_w).mean(dim=1)

    #fourth step : compute Lo and Le
        # Lo
    log_w = log_w.view(log_w.shape[0], 2, 2 ** (K + I0_start_level), 1)
    lower_level_term = train_function(log_w).mean(dim=1)
        #Le
    log_w = log_w.view(log_w.shape[0], 2 ** (K + I0_start_level + 1), 1)
    upper_level_term = train_function(log_w)
    
    #fifth step : compute DeltaK
    Delta_K = upper_level_term - lower_level_term
    
    #sixth step : compute the final estimator
    ml_ll_ss = I0 + Delta_K / K_pmf(K)

    return ml_ll_ss

In [82]:
ss(log_w, K, I0_start_level)

RuntimeError: shape '[25, 4, 1, 1]' is invalid for input of size 50